In [3]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import special
from toolbox import sphereical_transport, error
#inputs
n = 4
cells = 5
r = 1.0
scat_order = 0
sig_t = 30
sig_a = 3
q_0 = np.ones(cells) #p/cm^3-s
normalize = False
psi_incident = np.ones(int(n/2))/sig_a/2
accelerate = True
legendre_xs_mom = np.array([30])
#generate angular grid center points (roots of legendre polynomial of degree n) and the
#corresponding gauss quadrature weights
ang_cell_centers, w = np.polynomial.legendre.leggauss(n)
#generate angular cell edge values 
ang_cell_edges = np.zeros(n+1)
ang_cell_edges[0] = -1
for i in range(1,n+1):
    ang_cell_edges[i] = ang_cell_edges[i-1] + w[i-1]
#generate alpha coefficients
alpha = np.zeros(n+1)
for i in range(1,n+1):
    alpha[i] = alpha[i-1] - (2*w[i-1]*ang_cell_centers[i-1])
alpha[n] = 0
#generate beta coefficients
beta = np.zeros(n)
for i in range(n):
    beta[i] = (ang_cell_centers[i] - ang_cell_edges[i])/(ang_cell_edges[i+1] - ang_cell_edges[i])
#initialize sphereical transport class
sph_trans = sphereical_transport(q_0,legendre_xs_mom,cells,n,r,sig_t,sig_a,normalize,psi_incident,scat_order,ang_cell_centers,w,alpha,beta,accelerate)
#initial guess for source
ang_flux_0 = np.zeros(n)
for i in range(cells):
    sph_trans.update_scat_source_mom(i,ang_flux_0)
#generate initial phi 
phi_0 = np.zeros(cells)
epsilon = 1
#begin source iteration
index = 0
while epsilon > 1e-4:
    #compute psi at mu = -1 which will be the first angular inflow vector
    ang_inflow, origin_flux = sph_trans.starting_direction()
    #do the full domain sweep
    phi = sph_trans.sweep(ang_inflow,origin_flux)
    print('Index '+str(index))
    print(phi)
    if accelerate:
        sph_trans.DSA(phi,phi_0)
    index += 1
    if index > 99:
        print('Iteration limit exceeded')
        break
    epsilon = np.max(error.rel_change(phi,phi_0))
    phi_0 = phi
print('Final index and Phi')
print(index-1)
print(phi)
#compute balance tables
abs_rate = 0
for i in range(cells):
    abs_rate += phi[i]*sig_a*sph_trans.V(i)
reflection = 0.0
for ang in range(int(n/2),n):
    reflection += sph_trans.outflow[ang - int(n/2)]*ang_cell_centers[ang]*w[ang]*sph_trans.A(cells)
sink_rate = abs_rate + reflection
source_rate = 0
for i in range(cells):
    source_rate += q_0[i]*sph_trans.V(i)
for ang in range(0,int(n/2)):
    source_rate += sph_trans.psi_incident[ang]*-ang_cell_centers[ang]*w[ang]*sph_trans.A(cells)
bal = np.abs((source_rate - sink_rate)/source_rate)

[0.16666667 0.16666667]
Angular Flux
[[ 0.02019671  0.01104083  0.02682119 -0.0018647   0.05059795]
 [ 0.0238575   0.00901267  0.02646407  0.00397958  0.03311416]
 [ 0.01938521  0.01598059  0.01664251  0.01710826  0.01590782]
 [ 0.01482729  0.01765715  0.01626556  0.01670487  0.01685388]]
Index 0
[0.04038378 0.02628197 0.04309968 0.01891456 0.05543289]
Angular Flux
[[0.19629553 0.20297911 0.19381078 0.20961469 0.20268366]
 [0.194407   0.20416182 0.19398861 0.20640602 0.20810901]
 [0.19398248 0.20182801 0.19825263 0.20013853 0.213018  ]
 [0.19454765 0.20041545 0.19935558 0.19946675 0.2113646 ]]
Index 1
[0.38924302 0.40508705 0.39256305 0.40742702 0.41866463]
Angular Flux
[[0.3842698  0.3804015  0.38211671 0.40674907 0.29309512]
 [0.38200897 0.38408801 0.37694924 0.40881636 0.31213942]
 [0.38038608 0.38436461 0.37836167 0.40081434 0.33923565]
 [0.38046526 0.38356084 0.37968766 0.39695335 0.34973572]]
Index 2
[0.76320903 0.76689065 0.75756969 0.80756852 0.64840292]
Angular Flux
[[0.565618